
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/public/NER_UR.ipynb)

# **Recognize entities in Urdu text**

# **Colab Setup**

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

# **Start the Spark session**

In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

import pandas as pd
from pyspark.sql.types import StringType, IntegerType

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np


spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)
spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


# **🔎 For about model**


📌 **"uner_mk_140M_300d"**--> *This model uses Urdu word embeddings to find 7 different types of entities in Urdu text. It is trained using **urduvec_140M_300d** word embeddings, so please use the same embeddings in the pipeline.*

*   Persons-**PER**, Locations-**LOC**, Organizations-**ORG**, Dates-**DATE**, Designations-**DESIGNATION**, Times-**TIME**, Numbers-**NUMBER**..**





# **🔎Define Spark NLP pipeline**

In [3]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("urduvec_140M_300d", "ur") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

ner = NerDLModel.pretrained("uner_mk_140M_300d", "ur" ) \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler, 
        tokenizer,
        embeddings,
        ner,
        ner_converter])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
light_model = LightPipeline(pipelineModel)

urduvec_140M_300d download started this may take some time.
Approximate size to download 110.2 MB
[OK!]
uner_mk_140M_300d download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


# **🔎Sample text**

In [4]:
text = "کھیل کے لیے سب سے مناسب تاریخ سال کا آخر ہے کیونکہ سنہ دوہزارباءیس کے آغاز میں سرمائی اولمپکس ہوتی ہیں۔ بلیٹر نے بین الاقوامی اولمپک کمیٹی کو یقین دلایا کہ ورلڈ کپ اور سرمائی اولمپکس کی تاریخوں میں کوئی ٹکراؤ نہیں ہو گا، جو کہ فروری دوہزارباءیس میں قزاقستان یا چین میں ہونی ہیں۔ ﻿فٹبال کلب کی تاریخ میں سنیچر 25اکتوبر کو ریئل میڈرڈ اور بارسلونا کے درمیان اہم ترین میچ کھیلا جائے گا۔ اس میچ میں فٹبال کی دنیا کے دو بہترین کھلاڑی حصہ لیں گے۔ اس میچ کی ایک خاص بات اور بھی ہے کہ اس میں فٹبال کے عالمی کپ میں سب سے زیادہ گول کرنے والے کھلاڑی کے علاوہ تاریخ کے پانچ میں سے چار مہنگے ترین کھلاڑی بھی حصہ لیں گے۔"

# **🔎Run the pipeline**

In [5]:
light_result = light_model.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

result_df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                                   'begin': begin, 'end': end})
result_df

Before _validateStagesInputCols


,chunks,entities,begin,end
0,سنہ دوہزارباءیس,DATE,51,65
1,بلیٹر,PERSON,104,108
2,بین الاقوامی اولمپک کمیٹی,ORGANIZATION,113,137
3,فروری دوہزارباءیس,DATE,227,243
4,قزاقستان,LOCATION,249,256
5,چین,LOCATION,261,263
6,﻿فٹبال کلب,LOCATION,279,288
7,25اکتوبر,DATE,309,316
8,ریئل میڈرڈ,ORGANIZATION,321,330
9,بارسلونا,ORGANIZATION,336,343


# **🔎Visualize results**

In [6]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = light_result[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)